
# Nicholas Barinaga
# Envormental Impact of Spanish Agriculture
# CPSC 222 Spring 2024

In [25]:
# Package Importation
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np
from sklearn.feature_selection import r_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score

# Introduction

---
## Domain
I have a paticular Interest in nature and things that grow. I also have a familial connection with Spain. I knew I wanted to study something relating to nature and agriculture excited me. To scope down my project I decided to qualify the scope to a single country, Spain. I felt that while many things impact carbon emission and pollution in a more general sense, that agriculture and livestock would have a paticular affect in emissions. The correlation between livestock production and emissions is perhaps to obvious and so I decided to focus on crop production.


---
## The Datasets
- For data on Spains agricultural production a data set was sourced from The Food and Agriculture Organization of the United Nations
- This source was very well documented which greatly aided in understanding attributes
- This source provided a tool for specifcying the range and scope of data to download

    - https://www.fao.org/faostat/en/#data/QCL

- For data on Spains emissions data was sourced from Our World In Data 

    - https://github.com/owid/co2-data

---
## Hypothesis


-   The production of a certain crops in Spain during a year have a direct impact on Spains carbon emissions for that year. There is a crop type which has a greatest level of effect on carbon emissions so that as that crop is produced in higher quanities, carbon emissions grow as a direct result.

## Classification Task

1.  


# Data Analysis


--- 
## Agriculture Data

- When collectng data for Spains agricultre the FOA provided a very useful tool for selecting data. One could greatly vary the topic and scope of the data. This tool was very valuable as rather than having to refine a large and poorly organized data set, I was able iterativly change the parameters of the download to suit the needs of the project. 

- The inital idea was to look at individual crops. The number of crops produced however made this unweilding, so a final data set was settled upon that collected crop croups under the attribute title "Item".

    - An item groups all crops into categoreis like cereal, fruits, vegetables, oil producing crops, etc. 

    - This change afforded a more apporpriate scope and although graunality is lost, I believe more meaningful patterns will emerge.


- Below shows an example of some of the more important attributes. Of paticular interests are the "Item" "Year" "Element" and "Value" of an instance.
    - "Element" refers to what is being measured, in this case production which is the total domestic production of crop both in and outside the agricultural sector.

    - "Item" refers to the type of crop as discussed previously

    - "Year" refers to the year in which the production was measured

    - "Value" refers to the observed production value of a the crop type in that given year

    - "Unit" denotes the unit that value of production is measured in: here it is (T) tonnes

In [26]:
# Small Example of agriculture data
crops_groups = pd.read_csv('./data/spain_crops_groups.csv')
display(crops_groups.iloc[:3, 5:])

,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,Production,F1717,"Cereals, primary",1961,1961,t,7501313.0,A,Official figure,NaN
1,Production,F1717,"Cereals, primary",1962,1962,t,9274378.0,A,Official figure,NaN
2,Production,F1717,"Cereals, primary",1963,1963,t,9418967.0,A,Official figure,NaN


---
## Carbon Emissions Data

- This Data set was first found from an article from Our World In Data. The dataset was downloaded from Our World In Data's Co2 and Greenhouse Gas Emissions Database. This data has many attributes with many similar names. To better understand the attributes and importance to project documentation connected to the data set was consulted.

- While this database provideds information on many forms of emissions, carbon dioxide emissions was the focus of this project.  

- Below smalls a small portion of the data set. Of paticular interest are the "country", "year", and "co2_including_luc"
    - "country" refers to the country to which the instance belongs

    - "year" indicates the year in which the measurement was taken

    - "co2_including_luc" is define by the dataset documentation as "Annual CO₂ emissions including land-use change - Annual total emissions of carbon dioxide (CO₂), including land-use change, measured in million tonnes"
        - land-use change is the process by which humans shape and alter land to fit specific needs. Cleary agriculture is a major source of land use change as wilded areas are doemsticated to faciliate mass production of crops while other areas may be rewilded. These land use change events include deforestation for the purpose of agricultural production. This deforestation has a great impact on the earths carbon capturing ability. For these reasons this metric is the most pertanted to our study.




In [27]:
# Small Example of carbon emissons data with most attributes hidden
carbon_emissions = pd.read_csv('./data/carbon_emissions.csv')
display(carbon_emissions.iloc[41200:41204, :11])

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc
41200,Spain,1890,ESP,17749968.0,4.373549e+10,NaN,NaN,7.339,0.517,7.573,24.608
41201,Spain,1891,ESP,17834434.0,4.475052e+10,NaN,NaN,7.940,0.601,8.188,27.735
41202,Spain,1892,ESP,17916684.0,4.871361e+10,NaN,NaN,8.299,0.359,4.522,29.551
41203,Spain,1893,ESP,17996694.0,4.684359e+10,NaN,NaN,8.119,-0.180,-2.163,30.338


## Data Preperation

### Preparing the crop data

- Three primary areas of concern arose when preparing the crop data.
    
    1. The dataframe was formatted in way that made accessing values difficult and need to be reorganized and simplified.
    2. The value of production for Fibre Crops in the years of 1998 and 1999 were outliers
    3. The "Value" attribute was missing for three crop types between the years 2018-2022


- The data was also normalized.

## Issue 1

- Each instance had its "Element", "Item", "Year", "Value" as record belong to an instance. This meant that for every "Item" (crop type) there would be an instance of each year, thus there were multiple instances of every year. The ideal format would have only one instance for each year and each individual crop type would an attribute, in which the "Value" of the "Element" (production) would be stored. 
    
- The solution to this problem was to shift the axes of many parts of the data frame. The new data frame (example below)
    - For each column of a crop type the value represents the production value for that year in tonnes
    - While most of the crop types are self explanatory some need clarification
        - Oilcrops are crops that produce oil and there is a great variety of oil crops which can be harvested in different ways. For this reason many but not all oil producing crops are either aggregated to oil equivilant or cake equivilant. Oil cake is the resedue left over of a crop after extracting oil. Some oil producing crops such as dessert and table nuts are actually listed under the nuts category rather than as oil crops.
        - Examples of Roots and Tubers include: potatos, onions, carrots, turnips, etc.
        - Examples of Pulses include:Peas, Lentils, Beans, etc. 

In [28]:
# Crops data after making shift
crops_by_groups = pd.read_csv('./data/crops_by_group.csv')
display(crops_by_groups.iloc[:3, 1:])

,Year,Cereal,Citrus Fruits,Fibre,Fruit,Vegetables,Sugar Crops,"Oilcrops, Cake","Oilcrops, Oil",Pulses,Roots and Tubers,Treenuts
0,1961,7501313.0,1963100.0,125501.0,8056170.0,5135084.0,4668000.0,123085.0,454410.81,705227.0,5033925.0,337261.0
1,1962,9274378.0,1411000.0,135918.0,7842627.0,5088416.0,3843000.0,131119.0,408366.28,755270.0,4263018.0,237906.0
2,1963,9418967.0,2087300.0,113091.0,9093705.0,5315694.0,3048000.0,112455.0,728444.63,750300.0,5184266.0,269686.0


#### Issue 2

- The value produced for fibre crops in the years of 1998 and 1999 were tested using z scores and had a value greater than 3, thus they are outliers in the data.

- The solution here was to make a decision: the outliers would not be altered or removed from the data. The data is yearly measurements of the overall production of each crop category for every year. The nature of this data frame would mean that removing outliers would be removing enitre years worth of data. In addition this data is simply recorindg the observed production value of each crop, if in a specific year a disproportionate amount of one crop was produced that data isn't neccisarliy incorrect. Though it may cause issues later when developing the model the decision was made to keep these outliers.

#### Issue 3

- There were no values of production amount for Fibre crops, oil producing crops, or oil cake crops between the years 2018-2019

- To solve this issue two methods were developed. The first was effort was to train linear regression models for each category using the data from the five years before the missing data (2013-2017). Perhaps only using five years created an issues because the models were not producing satisfactory or "beleivable" results for the missing years' data.

- When this first solution failed to generate data that would "fit" another method was devloped. The second method was to take the mean of the five previous years for each crop type to generate a single year's production value. This process was repeated for every year. This means that the imputed data was being using to imput more data. 
    - 2018 was produced using 2013, 2014, 2015, 2016, 2017.
    - 2019 was produced using 2014, 2015, 2016, 2017, 2018 (the previously imputed val).
    - this was repeated for up to the year 2022.

- While it may be a cause for concern that imputed data was being used to imput other data, this method was perferable as it unlike the first method the outlier data for fibre crops (the years 1998-1999) had no impact on the imputations. The main reason this method was perfered was simply because the data produced did not greatly vary from the "real" instances before and after and continued the general trend that the "real" data was following.

 ### Preparing the emissions data

- This data set did not need much alteration before merging. Many of the changes were relating to scoping the data to size that would be perferable to use in conjunction with the crop data

    - All instaces pertaining to countries other than Spain were dropped.

    - Attributes with 30% or more of its data missing were removed.

    - Further all attributes outside of 'country','year', 'co2_including_luc', 'co2_including_luc_growth_abs', 'co2_including_luc_growth_prct' were dropped. The classifer for this study is co2_including_luc, however these other attributes were interesting variations on that data and at this point in the study I was still considering exploring at them.

    
    - Lastly all instances that belonged to years before 1961 were dropped. This was to match the emissions data with the available years of crop data
 